<h1>Run Enriched Polygons Through Trained OSM Completeness Model</h1>
<p>After enriching a collection of 250-m by 250-m grid cells, the output can be run through the trained OSM completeness model to produce predictions of OSM building footprint area.</p>

In [22]:
import json
import joblib
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape, box, mapping, Point, Polygon

In [3]:
#file containing enriched cells
with open(r'path_to_input_file','r') as data:
    x = json.load(data)

In [15]:
#create polygons for geodataframe
polygons=[]
for feature in x['features']:
    g = shape(feature['geometry']).buffer(0)
    polygons.append(g)
print(len(polygons))

442361


In [5]:
def returnFeatureVals(featureString, variable):
    return [x['properties'][featureString] for x in variable['features']]

In [16]:
applyFeatureDict = {
    'ndbi':returnFeatureVals('ndbi', x),
    'ndvi':returnFeatureVals('ndvi', x),
    'savi':returnFeatureVals('savi', x),
    'ui':returnFeatureVals('ui', x),
    'viirs':returnFeatureVals('viirs', x),
    'slope':returnFeatureVals('slope', x),
    'texture':returnFeatureVals('texture', x),
    'forest':returnFeatureVals('forest', x),
    'popFB':returnFeatureVals('popFB',x),
    'popWP':returnFeatureVals('popWP',x),
    'popWSF':returnFeatureVals('popWSF',x),
    'roads':returnFeatureVals('roads',x),
    'nodes':returnFeatureVals('nodes',x),
    'area':returnFeatureVals('area',x)
}
applyDF = pd.DataFrame.from_dict(applyFeatureDict)
applyGeoDF = gpd.GeoDataFrame(applyDF,crs = 4326, geometry=polygons)
applyGeoDF = applyGeoDF.fillna(0)

In [17]:
filename = r'path_to_trained_model\trained.sav'
model = joblib.load(filename)

In [18]:
applyFeatureDF = applyGeoDF[['ndbi','ndvi','savi','ui','viirs','slope','texture','forest','popFB','popWP','roads','nodes','popWSF']]
applyTargetDF = applyGeoDF['area']

In [19]:
y_apply = model.predict(applyFeatureDF)

In [20]:
for i,feature in enumerate(x['features']):
    area = x['features'][i]['properties']['area']
    #erase other properties for smaller output file
    feature['properties'] = {}
    #predicted OSM building footprint area
    feature['properties']['parea'] = y_apply[i]
    #actual mapped building footprint area
    feature['properties']['builtArea'] = area

In [21]:
with open(r'path_to_output\output.json','w') as f:
    json.dump(x,f)